In [1]:
import pandas as pd
import nltk
import spacy
import scipy
import os

In [8]:
def read_data(path="./dataset", key="key"):
    files = os.listdir(path)
    print(files)
    df = pd.concat([pd.read_csv(f"{path}/{file}") for file in files ], ignore_index=True)
    return df

In [ ]:
df = read_data()

['Books_5_part6.csv', 'Office_Products_5_part0.csv', 'Clothing_Shoes_and_Jewelry_5_part2.csv', 'Books_5_part5.csv', 'Clothing_Shoes_and_Jewelry_5_part0.csv', 'Industrial_and_Scientific_5_part0.csv', 'Clothing_Shoes_and_Jewelry_5_part1.csv', 'Books_5_part4.csv', 'Books_5_part0.csv', 'Books_5_part1.csv', 'Books_5_part3.csv', 'Kindle_Store_5_part0.csv', 'Appliances_5_part0.csv', 'Patio_Lawn_and_Garden_5_part0.csv', 'Books_5_part2.csv', 'Tools_and_Home_Improvement_5_part0.csv', 'Pet_Supplies_5_part0.csv', 'Arts_Crafts_and_Sewing_5_part0.csv', 'Movies_and_TV_5_part0.csv', 'AMAZON_FASHION_5_part0.csv', 'Software_5_part0.csv', 'Automotive_5_part0.csv', 'All_Beauty_5_part0.csv', 'Prime_Pantry_5_part0.csv', 'Sports_and_Outdoors_5_part0.csv', 'Toys_and_Games_5_part0.csv', 'Gift_Cards_5_part0.csv', 'Digital_Music_5_part0.csv', 'Home_and_Kitchen_5_part1.csv', 'Grocery_and_Gourmet_Food_5_part0.csv', 'Video_Games_5_part0.csv', 'Home_and_Kitchen_5_part0.csv', 'Magazine_Subscriptions_5_part0.csv', 'Lu

In [ ]:
df